<a href="https://colab.research.google.com/github/oddmanru/pills_detector/blob/main/pills_detector_custom_object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Pill Detector  
This code snippet is aimed at applying the trained custom object detector based on YOLOv5 engine to identify the pills. The custom object detector was converted from .pt to .onnx format by applying the conversion script. 

### Mount MyDrive  
It is a convenient measure that copies the stored files/scripts on Google cloud 
to local host

### Duplicate the cv2 4.7.0-dev   
The purpose of doing this is because the Colab needs the recompiled
version of OpenCV to support DNN on CUDA devices.

In [ ]:
# !cp drive/MyDrive/colab_modules/cv2_gpu/cv2.cpython-37m-x86_64-linux-gnu.so  /usr/lib/python3/dist-packages/cv2.cpython-37m-x86_64-linux-gnu.so
# !cp drive/MyDrive/colab_modules/cv2_gpu/python_38/cv2.cpython-38-x86_64-linux-gnu.so .

In [ ]:
# !python --version

In [ ]:
#  pip install opencv-python==4.5.5.64

In [ ]:
# !cd /usr/lib/python3/dist-packages

In [ ]:
# !ls cv2.cpython-3m-x86_64-linux-gnu.so

In [ ]:
# !mv /content/cv2.cpython-37m-x86_64-linux-gnu.so cv2.cpython-38m-x86_64-linux-gnu.so

In [ ]:
import cv2
cv2.__version__

'4.7.0-dev'

### Load the video clips   
Copy the video clips which need to be detected to the yolov5_pills directory

In [ ]:
!mkdir /content/yolov5_pills

mkdir: cannot create directory ‘/content/yolov5_pills’: File exists


In [ ]:
!cp /content/drive/MyDrive/computer_vision/yolo_v5_custom_object_detection/videos/pills_2.mp4 /content/yolov5_pills

In [ ]:
# !cp /content/drive/MyDrive/computer_vision/yolo_v5_custom_object_detection/videos/20221214_115546.mp4 /content/yolov5_pills

### Prepare the packages needed

In [ ]:
import os
os.sys.path.append("drive/MyDrive/custom_modules")
from cv_utils.delta_timer import Delta_timer
from unpack_utils.unpack_objects import UnpackObjects
from imutils.video import FPS
import numpy as np
# import cv2
# import dlib
import imutils
import time
import os
import logging

Enable logger 

In [ ]:
logger = logging.getLogger("__name__")
logging.basicConfig()
# Ctrl + M Z to redo 

### Load classes  
Load the classes.txt which contains the classes that this model can identify

In [ ]:
# Load labels
# labelsPath= os.path.join(args['yolo'], 'coco.names')
classPath = "drive/MyDrive/computer_vision/yolo_v5_custom_object_detection/classes.txt"
CLASSES= open(classPath, 'r').read().strip().split('\n') # Return a list
print(f"[testing] CLASSES length: {len(CLASSES)}")

# Create color list corresponding to the classes
COLORS = np.random.randint(0 ,255, size=(len(CLASSES), 3), dtype='uint8')

[testing] CLASSES length: 2


### Load pre-trained object detection model  
Load the trained custom object detector which is able to identify pills. This pills.onnx btw, was derived the pretrained yolov5s.pt 

In [ ]:

# Load YOLOv5 custom object detection model
print("[INFO] loading the object detection model...")
pills_path = "drive/MyDrive/computer_vision/yolo_v5_custom_object_detection/pills.onnx"

if os.path.isfile(pills_path):
  print('[INFO] loading YOLOv5 model from disk...')
  yoloV5_pills= cv2.dnn.readNet(pills_path) # ATTN: order of argument is different than mask-rcnn
  # Set up the cuda environment for yolo_v3 model
  yoloV5_pills.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
  yoloV5_pills.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
else:
  print("[ALERT] model does not exist...")

# Set up the cuda environment for yolo_v3 model
# yoloV5_pills.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
# yoloV5_pills.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)


[INFO] loading the object detection model...
[INFO] loading YOLOv5 model from disk...


In [ ]:
def blob_yoloV5(frame):
  '''
  This function applies a full-square sized mask
  on the given frame and makes blob image for dnn 
  network to detect objects
  '''
  h, w = frame.shape[:2]
  # Grab the maximum value of h and w
  _max = max(h, w)
  # Make a zeros mask
  mask_resized = np.zeros([_max, _max, 3], dtype="uint8")
  rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  # copy rgb to the mask
  mask_resized[0:h, 0:w] = rgb
  # Obtain the image shape before fed into blob
  resized_height, resized_width = mask_resized.shape[:2]

  blob = cv2.dnn.blobFromImage(mask_resized, 1.0/255, (640, 640), swapRB=False, crop=False)
  return resized_height, resized_width, blob


### Load Video clip

In [ ]:
# Load video clip
# video_path = "drive/MyDrive/computer_vision/mobileSSD_objects_tracking/videos/car_traffic.mp4"
video_path = "/content/yolov5_pills/pills_2.mp4"
camera = cv2.VideoCapture(video_path)
writer = None
# out_path = "drive/MyDrive/computer_vision/yolo_v5_deep_sort/out_videos/yolo5_deepsort_traffic_1024_np_itv_4_opti.mp4"
out_path = "/content/yolov5_pills/pills_2_done.mp4"

### Initialize variables and timers

In [ ]:
# Initialize the unpack objects class
yolov5_unpack = UnpackObjects(CLASSES)

# Initialize the trackeres and the corresponding labels
tracks = []
labels = []
idxs = []
boxes = []
confidences = []
classIDs = []
tracking_time = []
yolo_time = []
nms_time = []
new_detected_objects = []


# Interval time 
interval_time = 4
# tracker_counter = []


# Initialize timer for monitoring array operations
fReader_timer = Delta_timer()
blob_timer = Delta_timer()
yolo_timer = Delta_timer()
bb_finding_timer = Delta_timer() 
nms_timer = Delta_timer()
deepSort_timer = Delta_timer()
tracking_timer = Delta_timer()
tk_index_timer = Delta_timer()
index_timer = Delta_timer()
writing_timer = Delta_timer()

whole_timer = Delta_timer()

In [ ]:
# Initialize the DeepSORT tracker
# tracker = None
# if tracker is None:
#   deepSort_timer.start()
#   tracker = DeepSort(max_age=5, max_batch_size=128)
#   deepSort_timer.stop()
#   deepSort_timer.update()

In [ ]:
# Initialze the frame rate esitmator
fps = FPS().start()

frame_cnt = 0

### Main function

In [ ]:

while True:
	
  whole_timer.start()


  fReader_timer.start()
  (grabbed, frame) = camera.read()
  fReader_timer.stop()
  fReader_timer.update()

  # Check if video is at the end 
  if frame is None:
    break

  h, w = frame.shape[:2]
  if h > w: 
    frame = imutils.resize(frame, height = 1024)
  if w >= h:
    frame = imutils.resize(frame, width = 1024)

  # dlib tracker and dnn yolov5 need RGB order of the color channels
  rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  
  
  # Initialize the video writer
  if out_path is not None and writer is None:
    print("[video_writer] initializing video wirter...")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    print(f"[video_writer] {out_path}")
    writer = cv2.VideoWriter(out_path, fourcc, 30, (frame.shape[1], frame.shape[0]), True)
  

  # Prepare the input data before fed into the YOLOv5 model
  # blob_yolov5 function returns three scalar, i.e resized height, resized width and the 
  # blob that is going to be fed into the detector
  blob_timer.start()
  resized_height, resized_width, blob = blob_yoloV5(frame)
  yoloV5_pills.setInput(blob)
  blob_timer.stop()
  blob_timer.update()

  # YOLOv5 returns the real bboxes coords
  width_ratio = resized_width / 640
  height_ratio = resized_height / 640

  # Make the inference

  # (boxes, masks)= yoloV3.forward(['detection_out_final', 'detection_masks'])
  yolo_timer.start()
  layerOutputs= yoloV5_pills.forward()
  yolo_timer.stop()
  yolo_timer.update()

  # Return the candidates of bboxes
  # Loop over each of output layer
  # print(f"[yolov5 feedforward] layerOutputs shape: {layerOutputs.shape}")
  # print(f"[yolov5 feedforward] layerOutputs shape[0]: {layerOutputs.shape[0]}")
  # the shape of layerOutputs is 1 x 25200 x 85.
  detected_objects = layerOutputs[0]


  bb_finding_timer.start()
  boxes, confidences, classIDs = yolov5_unpack.unpack_numpy(detected_objects)
  bb_finding_timer.stop()
  bb_finding_timer.update()

  # apply non-maxima suppression to suppress weak, overlapping bounding
  # boxes. The cv's NMS takes trX, trY, and its w and h
  # confidence == 0.5, NMS thresh == 0.3
  # idxs= cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.3)
  nms_timer.start()

  # YOLOv5 has different NMS box arguments than YOLO v4
  # idxs= cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
  # idxs= cv2.dnn.NMSBoxes(boxes, confidences, 0.25, 0.45)
   idxs= cv2.dnn.NMSBoxes(boxes, confidences, 0.45, 0.45)
  nms_timer.stop()
  nms_timer.update()

  # index_timer.start()
  if len(idxs) > 0:

    index_timer.start()
    '''
    We need to create a new detection object here that contains
    bbs, confidence, and the detection classes ID to feed into 
    the new DeepSORT tracker in order to create a new tracking object 
    matrix
    The same format but the accurate ones that were done after NMS. 
    boxes, confidences, classIDs = yolov5_unpack.unpack_numpy(detected_objects) 

    # List of detections, each in tuples of ( [left,top,w,h] , confidence, detection_class)
    '''

    # Tracking these objects and draw the bboxes 
    for i, idx in enumerate(idxs.flatten()):
      # Obtain the final box in boxes that stores al the possible boxes
      (tlX, tlY)= (boxes[idx][0], boxes[idx][1])
      # ATTN! w, h here is the width and height of ROI boxes 
      (w, h )= (boxes[idx][2], boxes[idx][3])
      # Draw a color from COLOR array and concert it to list
      color= [int(c) for c in COLORS[classIDs[idx]]]
      # print(f"[INFO] color: {color}")

      # Align with the convention of dlib correlation tracker which 
      # takes tlX, tlY, brX, brY
      brX = tlX + w
      brY = tlY + h
     
      # Update the trackers and labels list
      # label = CLASSES[new_classIDs[i]]
      label = CLASSES[classIDs[idx]]
      labels.append(label)

      # Draw bounding box using the current coords 
      cv2.rectangle(frame, (tlX, tlY), (brX, brY), (0,255,0), 1)
      # Draw the title box
      t_width = int((brX - tlX)/2)
      cv2.rectangle(frame, (tlX, tlY-30), (tlX+t_width, tlY-5), color, -1)
      cv2.putText(frame, f"OD_{label}: #{i}", (tlX, tlY-10), cv2.FONT_HERSHEY_SIMPLEX, 0.55, (255,255,255), 1)

    # # frame? rgb? 
    # tracks = tracker.update_tracks(new_detected_objects, frame=rgb)
    
    # Index timer
    index_timer.stop()
    index_timer.update()

  writing_timer.start()
  writer.write(frame)
  writing_timer.stop()
  writing_timer.update()

  fps.update()

  whole_timer.stop()
  whole_timer.update()

IndentationError: ignored

In [ ]:
fps.stop()

### Create reports

In [ ]:
print(f"[INFO] elapsed time: {fps.elapsed():.2f}")
print(f"[INFO] approx FPS: {fps.fps():.2f}")
print("----------------------------")

print("[INFO] cleaning up...")
writer.release()
camera.release()

# Report on timers
fReader_report = fReader_timer.report("reading frame")
blob_report = blob_timer.report("blob transformer")
yolo_report = yolo_timer.report("yolo inference")
bb_finding_report = bb_finding_timer.report("unpack yolo")
nms_report = nms_timer.report("NMS")
# deepSort_report = deepSort_timer.report("DeepSORT intialization")
# tracking_report = tracking_timer.report("deepsort tracker")
# tk_indexing_report = tk_index_timer.report("tracker indexing")
index_report = index_timer.report("yolo indexing")
writing_report = writing_timer.report("writing frame")
whole_report = whole_timer.report("whole time")

In [ ]:
accum_time = fReader_timer.delta_sum + blob_timer.delta_sum + yolo_timer.delta_sum + bb_finding_timer.delta_sum + nms_timer.delta_sum + index_timer.delta_sum + writing_timer.delta_sum 
print(f"[accumulated time] : {accum_time:.4f}s")

In [ ]:
fps._numFrames

### Duplicate the artifest

In [ ]:
!cp /content/yolov5_pills/pills_1_done.mp4 /content/drive/MyDrive/computer_vision/yolo_v5_custom_object_detection/out_videos